In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [2]:
global pop_size
global parents
global epochs
global threshold
parents = []
pop_size = 40
epochs = 100
threshold = 0.735

In [3]:
df = pd.read_excel (r'Training_Data.xlsx')

In [4]:
df

,PatientId,EncounterId,DischargeDisposision,Gender,Race,DiabetesMellitus,ChronicKidneyDisease,Anemia,Depression,ChronicObstructivePulmonaryDisease,...,BetaBlockers,Diuretics,TotalMedicine,CardiacTroponin,Hemoglobin,SerumSodium,SerumCreatinine,BNP,NT-proBNP,ReadmissionWithin_90Days
0,4200412,199171333,Home,Male,White,DM,NaN,Anemia,NaN,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
1,4055894,26704337,Home,Male,White,DM,CKD,Anemia,Depression,COPD,...,1,5,8,0.0,0.00,0.0,1.540000,0.0,0.0,No
2,4867407,60388216,Home,Male,White,DM,CKD,Anemia,NaN,COPD,...,1,1,2,0.0,10.20,0.0,0.000000,0.0,0.0,No
3,4058064,274642265,Hospice - Home,Female,White,DM,NaN,Anemia,NaN,COPD,...,0,0,0,0.0,0.00,132.0,0.000000,0.0,0.0,No
4,4150623,70000001557327,SNF,Female,White,NaN,NaN,Anemia,NaN,COPD,...,0,0,0,0.0,7.26,0.0,0.000000,0.0,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,4152524,40004415567,Home Health,Female,White,DM,CKD,Anemia,NaN,COPD,...,2,3,6,0.0,0.00,0.0,1.076667,0.0,0.0,Yes
8477,4042227,14347947026,SNF,Male,White,DM,CKD,NaN,Depression,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
8478,4603405,67117733,Hospice,Female,White,NaN,NaN,NaN,NaN,NaN,...,1,0,1,0.0,0.00,0.0,0.000000,0.0,0.0,No
8479,4033677,68564389,Home,Female,White,DM,CKD,NaN,Depression,NaN,...,1,1,5,0.0,0.00,0.0,1.690000,0.0,0.0,Yes


In [5]:
for i in df:
    print(i, 'has', df[i].isnull().sum(), 'null values')

PatientId has 0 null values
EncounterId has 0 null values
DischargeDisposision has 0 null values
Gender has 0 null values
Race has 93 null values
DiabetesMellitus has 3857 null values
ChronicKidneyDisease has 3906 null values
Anemia has 3002 null values
Depression  has 5108 null values
ChronicObstructivePulmonaryDisease has 3954 null values
Age has 0 null values
ChronicDiseaseCount has 0 null values
LengthOfStay has 0 null values
EmergencyVisit has 0 null values
InpatientVisit has 0 null values
OutpatientVisit has 0 null values
TotalVisits has 0 null values
BMIMin has 0 null values
BMIMax has 0 null values
BMIMedian has 0 null values
BMIMean has 0 null values
BPDiastolicMin has 0 null values
BPDiastolicMax has 0 null values
BPDiastolicMedian has 0 null values
BPDiastolicMean has 0 null values
BPSystolicMin has 0 null values
BPSystolicMax has 0 null values
BPSystolicMedian has 0 null values
BPSystolicMean has 0 null values
TemperatureMin has 0 null values
TemperatureMax has 0 null value

In [6]:
def Preprocessing(df):
    df['Depression '] = df['Depression '].replace(to_replace = np.nan, value = "No Depression")
    df['Anemia'] = df['Anemia'].replace(to_replace = np.nan, value = "No Anemia")
    df['ChronicKidneyDisease'] = df['ChronicKidneyDisease'].replace(to_replace = np.nan, value = "No CKD")
    df['DiabetesMellitus'] = df['DiabetesMellitus'].replace(to_replace = np.nan, value = "No DM")
    df['ChronicObstructivePulmonaryDisease'] = df['ChronicObstructivePulmonaryDisease'].replace(to_replace = np.nan, value = "No COPD")
    df['Race'] = df['Race'].replace(to_replace = np.nan, value = df['Race'].mode()[0])
    df = df.astype({'EncounterId':'string'})
    
    return df

In [7]:
df = Preprocessing(df)

In [8]:
df

,PatientId,EncounterId,DischargeDisposision,Gender,Race,DiabetesMellitus,ChronicKidneyDisease,Anemia,Depression,ChronicObstructivePulmonaryDisease,...,BetaBlockers,Diuretics,TotalMedicine,CardiacTroponin,Hemoglobin,SerumSodium,SerumCreatinine,BNP,NT-proBNP,ReadmissionWithin_90Days
0,4200412,199171333,Home,Male,White,DM,No CKD,Anemia,No Depression,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
1,4055894,26704337,Home,Male,White,DM,CKD,Anemia,Depression,COPD,...,1,5,8,0.0,0.00,0.0,1.540000,0.0,0.0,No
2,4867407,60388216,Home,Male,White,DM,CKD,Anemia,No Depression,COPD,...,1,1,2,0.0,10.20,0.0,0.000000,0.0,0.0,No
3,4058064,274642265,Hospice - Home,Female,White,DM,No CKD,Anemia,No Depression,COPD,...,0,0,0,0.0,0.00,132.0,0.000000,0.0,0.0,No
4,4150623,70000001557327,SNF,Female,White,No DM,No CKD,Anemia,No Depression,COPD,...,0,0,0,0.0,7.26,0.0,0.000000,0.0,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,4152524,40004415567,Home Health,Female,White,DM,CKD,Anemia,No Depression,COPD,...,2,3,6,0.0,0.00,0.0,1.076667,0.0,0.0,Yes
8477,4042227,14347947026,SNF,Male,White,DM,CKD,No Anemia,Depression,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
8478,4603405,67117733,Hospice,Female,White,No DM,No CKD,No Anemia,No Depression,No COPD,...,1,0,1,0.0,0.00,0.0,0.000000,0.0,0.0,No
8479,4033677,68564389,Home,Female,White,DM,CKD,No Anemia,Depression,No COPD,...,1,1,5,0.0,0.00,0.0,1.690000,0.0,0.0,Yes


In [9]:
label_encoder = preprocessing.LabelEncoder()
df['Anemia'] = label_encoder.fit_transform(df['Anemia'])
df['DiabetesMellitus'] = label_encoder.fit_transform(df['DiabetesMellitus'])
df['ChronicKidneyDisease'] = label_encoder.fit_transform(df['ChronicKidneyDisease'])
df['Depression '] = label_encoder.fit_transform(df['Depression '])
df['ChronicObstructivePulmonaryDisease'] = label_encoder.fit_transform(df['ChronicObstructivePulmonaryDisease'])
df['DischargeDisposision'] = label_encoder.fit_transform(df['DischargeDisposision'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Race'] = label_encoder.fit_transform(df['Race'])
df['ReadmissionWithin_90Days'] = label_encoder.fit_transform(df['ReadmissionWithin_90Days'])
df['EncounterId'] = label_encoder.fit_transform(df['EncounterId'])

In [10]:
y = df['ReadmissionWithin_90Days']
x = df.drop('ReadmissionWithin_90Days', axis = 1)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
clf = GaussianNB()
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
prior_acc = accuracy_score(y_test, pred)
print(prior_acc)

0.41131408367707717


In [12]:
chromosomes = []
total_chromo = []
for j in range(pop_size):
    for i in range(len(x.columns)):
        total_chromo.append(random.randint(0, 1))
    chromosomes.append(total_chromo)
    total_chromo = []

In [13]:
cols = x.columns
cols

Index(['PatientId', 'EncounterId', 'DischargeDisposision', 'Gender', 'Race',
       'DiabetesMellitus', 'ChronicKidneyDisease', 'Anemia', 'Depression ',
       'ChronicObstructivePulmonaryDisease', 'Age', 'ChronicDiseaseCount',
       'LengthOfStay', 'EmergencyVisit', 'InpatientVisit', 'OutpatientVisit',
       'TotalVisits', 'BMIMin', 'BMIMax', 'BMIMedian', 'BMIMean',
       'BPDiastolicMin', 'BPDiastolicMax', 'BPDiastolicMedian',
       'BPDiastolicMean', 'BPSystolicMin', 'BPSystolicMax', 'BPSystolicMedian',
       'BPSystolicMean', 'TemperatureMin', 'TemperatureMax',
       'TemperatureMedian', 'TemperatureMean', 'HeartRateMin', 'HeartRateMax',
       'HeartRateMedian', 'HeartRateMean', 'PulseRateMin', 'PulseRateMax',
       'PulseRateMedian', 'PulseRateMean', 'RespiratoryRateMin',
       'RespiratoryRateMax', 'RespiratoryRateMedian', 'RespiratoryRateMean',
       'ACEInhibitors', 'ARBs', 'BetaBlockers', 'Diuretics', 'TotalMedicine',
       'CardiacTroponin', 'Hemoglobin', 'SerumSod

In [14]:
def Fitness_function(chromosomes):
    a = []
    pop = []
    for index, i in enumerate(chromosomes):
        for index2, j in enumerate(i):
            if (j == 1):
                a.append(cols[index2])
            else:
                pass
        pop.append(a)
        a = []
    acc = []
    for i in range(len(pop)):
        x_train, x_test, y_train, y_test = train_test_split(df[pop[i]], y, test_size = 0.2, random_state = 42)
        clf = GaussianNB()
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        acc.append(accuracy_score(y_test, pred))
    return acc

In [15]:
acc = Fitness_function(chromosomes)

In [16]:
acc

[0.7124337065409546,
 0.3971714790807307,
 0.690041249263406,
 0.7041838538597525,
 0.699469652327637,
 0.704773129051267,
 0.7136122569239834,
 0.6894519740718916,
 0.7171479080730702,
 0.4667059516794343,
 0.6888626988803771,
 0.7071302298173247,
 0.690041249263406,
 0.3635827931644078,
 0.7112551561579258,
 0.30642309958750735,
 0.713022981732469,
 0.3164407778432528,
 0.7018267530936948,
 0.5486152032999411,
 0.41013553329404834,
 0.6988803771361226,
 0.6723629935179729,
 0.33588685916322925,
 0.41131408367707717,
 0.357100766057749,
 0.3264584560989982,
 0.4236888626988804,
 0.3276370064820271,
 0.629345904537419,
 0.30583382439599294,
 0.7118444313494402,
 0.6552740129640542,
 0.699469652327637,
 0.38243959929286975,
 0.7024160282852092,
 0.3235120801414261,
 0.7041838538597525,
 0.7018267530936948,
 0.708898055391868]

In [17]:
def velocity(w, v, c1, r1, c2, r2, pb, gb, x):
    return (w*v) + (c1*r1)*(pb - x) - (c2*r2)*(gb.reshape(-1, 1)-x)

In [18]:
v = np.random.rand(56, 40)

In [19]:
chromosomes = np.transpose(chromosomes)

In [20]:
pbest = chromosomes
pbest_obj = acc
gbest = pbest[:, np.array(pbest_obj).argmax()]
gbest_obj = np.argmax(pbest_obj)

In [21]:
epochs = 100
w = 0.8
r1 = np.random.rand()
r2 = np.random.rand()
c1 = 2
c2 = 2

for i in range(epochs):
    v = velocity(w, v, c1, r1, c2, r2, pbest, gbest, chromosomes)
    chromosomes = chromosomes + v
    
    for index, i in enumerate(chromosomes):
        for index2, j in enumerate(i):
            if (j >= 0.5):
                i[index2] = 1
            else:
                i[index2] = 0
            
    obj = Fitness_function(np.transpose(chromosomes))
    pbest[:, (pbest_obj >= obj)] = chromosomes[:, (pbest_obj >= obj)]
    
    pbest_obj = np.array([pbest_obj, obj]).max(axis = 0)
    gbest = pbest[:, pbest_obj.argmax()]
    gbest_obj = pbest_obj.max()

In [22]:
gbest_obj

0.7171479080730702

In [29]:
print("Accuracy prior to POS was", prior_acc, "and after POS was", gbest_obj, "\nSo there was a", gbest_obj - prior_acc, "improvement in the models accuracy")

Accuracy prior to POS was 0.41131408367707717 and after POS was 0.7171479080730702 
So there was a 0.305833824395993 improvement in the models accuracy
